<a href="https://colab.research.google.com/github/MaterialI/LLAMA-3.1-8b-financial-news-fine-tune/blob/main/LoraFinetune_LLAMA8b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U bitsandbytes
!pip install wandb
!pip install peft
!pip install trl
!pip install datasets
!pip install --upgrade transformers
# !pip install -q --upgrade transformers autoawq accelerate


In [ ]:
!pip install --upgrade autoawq

In [ ]:
import os
from google.colab import userdata
# Note: `userdata.get` is a Colab API. If you're not using Colab, set the env
# vars as appropriate for your system.
os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")

In [ ]:
import torch
from awq import AutoAWQForCausalLM
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "hugging-quants/Meta-Llama-3.1-70B-Instruct-AWQ-INT4"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoAWQForCausalLM.from_pretrained(
  model_id,
  torch_dtype=torch.float16,
  low_cpu_mem_usage=True,
  device_map="auto",
)


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
from transformers import BitsAndBytesConfig
base_model = "meta-llama/Meta-Llama-3.1-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(base_model)


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
    # load_in_8bit_fp32_cpu_offload=True,
)
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    device_map="auto",
    torch_dtype="float16",
    quantization_config=bnb_config,

)
# pipe = pipeline(
#     "text-generation",
#     model=model,
#     tokenizer=tokenizer,
#     torch_dtype=torch.float16,
#     device_map="auto",
#     # max_seq_length=16800,
# )
tokenizer.pad_token_id = tokenizer.eos_token_id

In [ ]:
messages = [{"role": "user", "content": "Give number of words in the following piece of news -> FDA grants Fast Track designation for nab-sirolimus for the treatment of adult and adolescent patients with Tuberous Sclerosis Complex 1 and 2 ( TSC1  and  TSC2 ) alterationsLOS ANGELES, March  31, 2022  (GLOBE NEWSWIRE) -- Aadi Bioscience, Inc. (Nasdaq: AADI), a biopharmaceutical company focusing on precision therapies for genetically-defined cancers with alterations in mTOR pathway genes, today announced the commencement of dosing in PRECISION 1, a Phase 2 registrational trial of nab-sirolimus in patients 12 years and older with solid tumors with pathogenic inactivating alterations in TSC1 and TSC2 genes. The FDA granted Aadi Fast Track designation to evaluate nab-sirolimus for this patient population. Fast Track designation is an FDA process designed to facilitate the development, and expedite the review of, medicines to treat serious conditions and fill unmet medical need. Neil Desai, Ph.D., Founder, Chief Executive Officer, and President of Aadi, stated  Following the approval and commercial launch of FYARRO  (also known as nab-sirolimus) in advanced malignant PEComa, and initial demonstration of single agent activity outside of PEComa in patients with inactivating alterations in TSC1 or TSC2, we are now pursuing a tumor-agnostic indication for nab-sirolimus in patients with inactivating alterations in TSC1 or TSC2 following feedback on the study design from the U.S. Food and Drug Administration (FDA). These alterations are estimated to occur in over 10,000 advanced cancer patients per year, representing one of the larger targeted oncology opportunities.  Dr. Desai continued,  We believe the initiation of this registrational trial highlights our team s ability to execute on drug development milestones and demonstrates meaningful progress towards expanding our nab-sirolimus program and establishing Aadi as a multi-indication, precision oncology company.  The PRECISION 1 trial is a multi-center, open-label, tumor-agnostic registrational clinical trial of nab-sirolimus. Designed as a  basket trial , the study will evaluate approximately 60 mTOR inhibitor na ve patients in each of two independent study arms (i.e., approximately 120 patients in total) comprised of patients with solid tumors harboring pathogenic inactivating alterations in either TSC1 or TSC2 genes. Preliminary data from PRECISION 1 are expected in the first half of 2023.  We are excited to have initiated dosing in our PRECISION 1 clinical trial,  stated Loretta Itri, M.D., Chief Medical Officer of Aadi.  There are currently no approved treatment options for patients with TSC1 or TSC2 alterations, and this trial is designed to evaluate the efficacy, safety and tolerability of nab-sirolimus to treat patients with tumors driven by those alterations. We believe that the early data we presented at ASCO 2021 from our Expanded Access Program for FYARRO suggested activity of nab-sirolimus in this patient population. In the PRECISION 1 trial, the two arms will be independently evaluable in order to separately assess therapeutic activity of nab-sirolimus in tumors harboring either TSC1 or TSC2 inactivating alterations.  About Aadi Bioscience  Aadi is a biopharmaceutical company focused on precision therapies for genetically-defined cancers. Aadi s primary goal is to bring transformational therapies to cancer patients with mTOR pathway driver alterations where other mTOR inhibitors have not or cannot be effectively exploited due to problems of pharmacology, effective drug delivery, safety, or effective targeting to the disease site. In November 2021, Aadi received FDA approval for FYARRO  (sirolimus protein-bound particles for injectable suspension)(albumin-bound) for the treatment of adult patients with locally advanced unresectable or metastatic malignant perivascular epithelioid cell tumor (PEComa), and in February 2022 Aadi announced the commercial launch of FYARRO in this indication. FYARRO is an mTOR inhibitor bound to human albumin that has demonstrated significantly higher tumor accumulation, greater mTOR target suppression, and increased tumor growth inhibition over other mTOR inhibitors in preclinical models. Based on data from Aadi s Phase 2 registrational study (the Advanced Malignant PEComa Trial, or  AMPECT ) with FYARRO and following discussions with the FDA about other emerging data with FYARRO, Aadi has initiated PRECISION 1, a tumor-agnostic registrational trial in mTOR inhibitor-na ve solid tumors harboring TSC1 or TSC2 inactivating alterations. Aadi also has ongoing studies to evaluate dosing of FYARRO in combination regimens. More information on Aadi s development pipeline is available on the Aadi website at www.aadibio.com. About FYARRO  FYARRO is an mTOR inhibitor indicated for the treatment of adult patients with locally advanced unresectable or metastatic malignant perivascular epithelioid cell tumor (PEComa). Important Safety Information Contraindication FYARRO is contraindicated in patients with a history of severe hypersensitivity to sirolimus, other rapamycin derivatives, or albumin. Warnings and Precautions Stomatitis Stomatitis, including mouth ulcers and oral mucositis, occurred in 79% of patients treated with FYARRO, including 18% Grade 3. Stomatitis was most often first reported within 8 weeks of treatment. Based on the severity of the adverse reaction, withhold, resume at reduced dose, or permanently discontinue FYARRO. Myelosuppression FYARRO can cause myelosuppression including anemia, thrombocytopenia and neutropenia. Anemia occurred in 68% of patients; 6% were Grade 3. Thrombocytopenia and neutropenia occurred in 35% of patients each. Obtain blood counts at baseline and every 2 months for the first year of treatment and every 3 months thereafter, or more frequently if clinically indicated. Based on the severity of the adverse reaction, withhold, resume at reduced dose, or permanently discontinue FYARRO. Infections FYARRO can cause infections. Infections such as urinary tract infections (UTI), upper respiratory tract infections and sinusitis occurred in 59% of patients. Grade 3 infections occurred in 12% of patients, including a single case each of a UTI, pneumonia, skin, and abdominal infections. Monitor patients for infections, including opportunistic infections. Based on the severity of the adverse reaction, withhold, resume at reduced dose, or permanently discontinue FYARRO. Hypokalemia FYARRO can cause hypokalemia. Hypokalemia occurred in 44% of patients including 12% Grade 3 events. Monitor potassium levels prior to starting FYARRO and implement potassium supplementation as medically indicated. Based on the severity of the adverse reaction, withhold, resume at reduced dose, or permanently discontinue FYARRO. Hyperglycemia FYARRO can cause hyperglycemia. Hyperglycemia occurred in 12% of patients treated with FYARRO, all of which were Grade 3 events. Monitor fasting serum glucose prior to starting FYARRO. During treatment, monitor serum glucose every 3 months in non-diabetic patients, or as clinically indicated. Monitor serum glucose more frequently in diabetic patients. Based on the severity of the adverse reaction, withhold, resume at reduced dose, or permanently discontinue FYARRO. Interstitial Lung Disease / Non-Infectious Pneumonitis FYARRO can cause interstitial lung disease (ILD) / non-infectious pneumonitis. ILD / non-infectious pneumonitis occurred in 18% of patients treated with FYARRO, of which all were Grades 1 and 2. Based on the severity of the adverse reaction, withhold, reduce the dose, or permanently discontinue FYARRO. Hemorrhage FYARRO can cause serious and sometimes fatal hemorrhage. Hemorrhage occurred in 24% of patients treated with FYARRO, including Grade 3 and Grade 5 events in 2.9% of patients each. Monitor patients for signs and symptoms of hemorrhage. Based on the severity of adverse reaction, withhold, resume at reduced dose, or permanently discontinue FYARRO. Hypersensitivity Reactions FYARRO can cause hypersensitivity reactions. Hypersensitivity reactions, including anaphylaxis, angioedema, exfoliative dermatitis, and hypersensitivity vasculitis have been observed with administration of the oral formulation of sirolimus. Hypersensitivity reactions including anaphylaxis have been observed with human albumin administration. Monitor patients closely for signs and symptoms of infusion reactions during and following each FYARRO infusion in a setting where cardiopulmonary resuscitation medication and equipment are available. Monitor patients for at least 2 hours after the first infusion and as clinically needed for each subsequent infusion. Reduce the rate, interrupt infusion, or permanently discontinue FYARRO based on severity and institute appropriate medical management as needed. Embryo-Fetal Toxicity Based on animal studies and the mechanism of action, FYARRO can cause fetal harm when administered to a pregnant woman. In animal studies, mTOR inhibitors caused embryo-fetal toxicity when administered during the period of organogenesis at maternal exposures that were equal to or less than human exposures at the recommended lowest starting dose. Advise pregnant women of the potential risk to a fetus. Advise females of reproductive potential to avoid becoming pregnant and to use effective contraception while using FYARRO and for 12 weeks after the last dose. Male Infertility Azoospermia or oligospermia may be observed in patients treated with FYARRO. FYARRO is an anti-proliferative drug and affects rapidly dividing cells such as germ cells. Immunizations and Risks Associated with Live Vaccines No studies in conjunction with immunization have been conducted with FYARRO. Immunization during FYARRO treatment may be ineffective. Update immunizations according to immunization guidelines prior to initiating FYARRO, if possible. Immunization with live vaccines is not recommended during treatment and avoid close contact with those who have received live vaccines while on FYARRO. The interval between live vaccinations and initiation of FYARRO should be in accordance with current vaccination guidelines for patients on immunosuppressive therapies. Risk of Transmission of Infectious Agents with Human Albumin FYARRO contains human albumin, a derivative of human blood. Human albumin carries only a remote risk of transmission of viral diseases because of effective donor screening and product manufacturing processes. A theoretical risk for transmission of Creutzfeldt-Jakob Disease (CJD) also is considered extremely remote. No cases of transmission of viral diseases or CJD have ever been associated with albumin. Adverse Reactions Adverse Reactions in PEComa The most common adverse reactions ( 30%) were stomatitis in 27 (79%) patients; fatigue and rash in 23 (68%) patients each; infection in 20 (59%) patients; nausea and edema in 17 (50%) patients each; diarrhea, musculoskeletal pain and decreased weight in 16 (47%) patients each; decreased appetite in 15 (44%) patients; cough in 12 (35%) patients; and vomiting and dysgeusia in 11 (32%) patients each. Laboratory Abnormalities in PEComa The most common Grade 3 to 4 laboratory abnormalities ( 6%) were decreased lymphocytes in 7 (21%) patients; increased glucose and decreased potassium in 4 (12%) patients each; decreased phosphate in 3 (9%) patients; and decreased hemoglobin and increased lipase in 2 (6%) patients each. Dosage interruptions Dose interruptions of FYARRO due to an adverse reaction occurred in 22 (65%) patients. Adverse reactions which required dosage interruption in >5% of patients included stomatitis in 6 (18%) patients, pneumonitis in 5 (15%) patients, anemia in 3 (9%) patients, and dehydration, dermatitis acneiform, and thrombocytopenia in 2 (6%) patients each. Dose reduction Dose reductions of FYARRO due to an adverse reaction occurred in 12 (35%) patients. Adverse reactions which required dose reductions in > 5% of patients included stomatitis and pneumonitis in 3 (9%) patients each. Drug Interactions Reduce the dosage of FYARRO to 56 mg/m2 when used concomitantly with a moderate or weak cytochrome P-450 3A4 (CYP3A4) inhibitor. Avoid concomitant use with drugs that are strong CYP3A4 and/or P-glycoprotein (P-gp) inhibitors and inducers and with grapefruit and grapefruit juice. Use in Specific Populations Pregnancy Based on the mechanism of action and findings in animals, FYARRO can cause fetal harm when administered to a pregnant woman. Advise females of the potential risk to a fetus and to avoid becoming pregnant while receiving FYARRO. Lactation Sirolimus is present in the milk of lactating rats. There is potential for serious adverse effects from sirolimus in breastfed infants based on mechanism of action. Because of the potential for serious adverse reactions in breastfed infants from FYARRO, advise women not to breastfeed during treatment with FYARRO and for 2 weeks after the last dose. Females and Males of Reproductive Potential FYARRO can cause fetal harm when administered to a pregnant woman. Verify the pregnancy status of females of reproductive potential prior to starting treatment with FYARRO. Advise females of reproductive potential to use effective contraception and avoid becoming pregnant during treatment with and for at least twelve weeks after the last dose of FYARRO. Advise males with female partners of reproductive potential to use effective contraception and avoid fathering a child during treatment with FYARRO and for at least twelve weeks after the last dose of FYARRO. Although there are no data on the impact of FYARRO on fertility, based on available clinical findings with oral formulation of sirolimus and findings in animals, male and female fertility may be compromised by the treatment with FYARRO. Pediatric The safety and effectiveness of FYARRO in pediatric patients have not been established. Geriatric Use Of the 34 patients treated with FYARRO, 44% were 65 years of age and older, and 6% were 75 years of age and older. Clinical studies of FYARRO did not include sufficient numbers of patients aged 65 and over to determine whether they respond differently from younger patients. Hepatic Impairment FYARRO is not recommended for use in patients with severe hepatic impairment. Reduce FYARRO dosage in patients with mild or moderate hepatic impairment. Full prescribing information can be found here. Forward-Looking Statements Aadi cautions you that certain statements included in this press release that are not a description of historical facts are forward-looking statements. These statements are based on Aadi s current beliefs and expectations. Forward-looking statements include statements regarding: the expectations regarding the beneficial characteristics, safety, efficacy and therapeutic effects of FYARRO, our plans related to further development and manufacturing of FYARRO, the timing of additional clinical trials, including the registrational trial in patients harboring TSC1 and TSC2 inactivating alterations, the timing or likelihood of regulatory filings and approvals of FYARRO, including in potential additional indications and potential filings in additional jurisdictions, anticipated reception of FYARRO in the physician community, and the sufficiency of our existing capital resources to fund our future operating expenses and capital expenditure requirements. Actual results could differ materially from those anticipated in such forward-looking statements as a result of these risks and uncertainties, which include, without limitation: uncertainties associated with the clinical development and regulatory approval of FYARRO, including potential delays in the commencement, enrollment and completion of clinical trials for additional indications; the risk that interim results of clinical trials may not be reproduced and do not necessarily predict final results; the risk that one or more of the clinical outcomes may materially change as patient enrollment continues, following more comprehensive reviews of the data, and as more patient data become available; the risk that unforeseen adverse reactions or side effects may occur in the course of commercializing, developing and testing FYARRO; risks related to Aadi s estimates regarding future expenses, capital requirements and need for additional financing; and risks related to the impact of the COVID-19 outbreak on Aadi s operations, the biotechnology industry and the economy generally. Additional risks and uncertainties that could cause actual outcomes and results to differ materially from those contemplated by the forward-looking statements are included under the caption ""Risk Factors"" in Aadi s Form 10-K filed on March 17, 2022 and elsewhere in Aadi s reports and other documents that Aadi has filed, or will file, with the SEC from time to time and available at www.sec.gov. All forward-looking statements in this press release are current only as of the date hereof and, except as required by applicable law, Aadi undertakes no obligation to revise or update any forward-looking statement, or to make any other forward-looking statements, whether as a result of new information, future events or otherwise. All forward-looking statements are qualified in their entirety by this cautionary statement. This caution is made under the safe harbor provisions of the Private Securities Litigation Reform Act of 1995. FYARRO  is a trademark of Aadi Bioscience, Inc. Contacts Investors:  Irina KofflerLifeSci Advisors LLCikoffler@lifesciadvisors.com Media: Darren Opland, Ph.D.LifeSci Communicationsdarren@lifescicomms.com"}]
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto",
    # max_seq_length=16800,
)
prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
outputs = pipe(prompt, max_new_tokens=150, do_sample=True)
print(outputs[0]["generated_text"])



In [ ]:
!mkdir data
!unzip data_processed_reaction_neutral.zip -d data

In [ ]:
import pandas as pd
df = pd.read_csv("./data/data_processed_reaction.csv")
print(df.shape)
df.head()
df.dropna(subset = ["sentiment", "news"],inplace=True)
df.to_csv("./data/data_processed_reaction.csv", index=False)

In [ ]:
def generate_prompt(data_point):
    return f"""
            Classify the news if it has Neutral, Positive or Negative sentiment, and return the answer as the corresponding sentiment label.
text: {data_point["news"]}
label: {data_point["sentiment"]}""".strip()

def generate_test_prompt(data_point):
    return f"""
            Classify the news if it has Neutral, Positive or Negative sentiment, and return the answer as the corresponding sentiment label.
text: {data_point["news"]}
label: """.strip()


In [ ]:
def assignLabel(x):
  if x == 1:
    return "Positive"
  elif x == 0:
    return "Neutral"
  else:
    return "Negative"
from datasets import load_dataset, Dataset, DatasetDict
dataset = load_dataset("csv", data_dir = "./data")
print(dataset)
dataset = dataset.remove_columns(["Unnamed: 0", "Link", "Impact", "Return", "Sentiment", "Tag"])
print(dataset)

dataset.shuffle(seeds=[12,14,16,18,42])
dataset.set_format("pandas")
dataset = dataset['train'][:]
dataset.drop_duplicates(inplace=True)

#----------------------------------------------
#limit the number of values in the dataset
dataset = dataset.iloc[:100] #to change
#----------------------------------------------


# print(dataset)
dataset = dataset.reset_index()[["news", "sentiment"]]
dataset["sentiment"] = dataset["sentiment"].apply(assignLabel)

# print(dataset.iloc[0])
dataset = Dataset.from_pandas(dataset)

train_test_valid = dataset.train_test_split(test_size=0.2, seed=69)

#test and val data
test_val_data = train_test_valid["test"].to_pandas()

test_val_data["text"] = test_val_data.apply(generate_test_prompt, axis=1)
test_val_data.drop(columns= ["news"], inplace=True)
#switch places of columns
test_val_data = test_val_data[["text", "sentiment"]]
#train data
train_data = train_test_valid["train"].to_pandas()

train_data["text"] = train_data.apply(generate_prompt, axis=1)
train_data.drop(columns= ["news"], inplace=True)
train_data = train_data[["text", "sentiment"]]


# convert both test, train
test_val_data = Dataset.from_pandas(test_val_data)
train_data = Dataset.from_pandas(train_data)


#split test into test, val
test_valid = test_val_data.train_test_split(test_size=0.5, seed = 69)
data = DatasetDict({
    "train": train_data,
    "test": test_valid["test"],
    "validation": test_valid["train"]
})
data

In [ ]:
evalDf = pd.read_csv("./data/data_processed_reaction.csv")
# for every row read column "news" and count number of words
evalDf["word_count"] = evalDf["news"].apply(lambda x: len(x.split()))
print(evalDf["word_count"].describe())

In [ ]:
statTrainDistr = data["train"].to_pandas()
print(statTrainDistr["sentiment"].value_counts())


In [ ]:

from transformers import AutoTokenizer,AutoModelForCausalLM,pipeline
import torch
from transformers import BitsAndBytesConfig

base_model = "meta-llama/Meta-Llama-3.1-8B-Instruct"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
)
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    device_map="auto",
    torch_dtype="float16",
    quantization_config=bnb_config,
)

model.config.use_cache = False
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(base_model)

tokenizer.pad_token_id = tokenizer.eos_token_id

In [ ]:
from tqdm import tqdm
def predict(test, model, tokenizer):
    y_pred = []
    categories = ["Positive","Neutral", "Negative"]

    for i in tqdm(range(len(test))):
        prompt = test.iloc[i]["text"]
        print(prompt)
        pipe = pipeline(task="text-generation",
                        model=model,
                        tokenizer=tokenizer,
                        max_new_tokens = 150,
                        temperature = 0.1,

                        )

        result = pipe(prompt)
        answer = result[0]['generated_text'].split("label:")[-1].strip()
        # print(answer)
        # Determine the predicted category
        for category in categories:
            if category.lower() in answer.lower():
                y_pred.append(category)
                break
        else:
            y_pred.append("none")

    return y_pred

y_pred = predict(data["test"].to_pandas(), model, tokenizer)

In [ ]:
y_pred

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
def evaluate(y_true, y_pred):
    labels = ["Positive","Neutral", "Negative"]
    mapping = {label: idx for idx, label in enumerate(labels)}

    def map_func(x):
        return mapping.get(x, -1)  # Map to -1 if not found, but should not occur with correct data

    y_true_mapped = np.vectorize(map_func)(y_true)
    y_pred_mapped = np.vectorize(map_func)(y_pred)

    # Calculate accuracy
    accuracy = accuracy_score(y_true=y_true_mapped, y_pred=y_pred_mapped)
    print(f'Accuracy: {accuracy:.3f}')

    # Generate accuracy report
    unique_labels = set(y_true_mapped)  # Get unique labels

    for label in unique_labels:
        label_indices = [i for i in range(len(y_true_mapped)) if y_true_mapped[i] == label]
        label_y_true = [y_true_mapped[i] for i in label_indices]
        label_y_pred = [y_pred_mapped[i] for i in label_indices]
        label_accuracy = accuracy_score(label_y_true, label_y_pred)
        print(f'Accuracy for label {labels[label]}: {label_accuracy:.3f}')

    # Generate classification report
    class_report = classification_report(y_true=y_true_mapped, y_pred=y_pred_mapped, target_names=labels, labels=list(range(len(labels))))
    print('\nClassification Report:')
    print(class_report)

    # Generate confusion matrix
    conf_matrix = confusion_matrix(y_true=y_true_mapped, y_pred=y_pred_mapped, labels=list(range(len(labels))))
    print('\nConfusion Matrix:')
    print(conf_matrix)

evaluate(data["test"].to_pandas()["sentiment"], y_pred)

In [ ]:
import bitsandbytes as bnb

def find_all_linear_names(model):
    cls = bnb.nn.Linear4bit
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names:  # needed for 16 bit
        lora_module_names.remove('lm_head')
    return list(lora_module_names)
modules = find_all_linear_names(model)
modules

In [ ]:
import torch
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from trl import SFTTrainer
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType

base_model = "meta-llama/Meta-Llama-3.1-8B-Instruct"

output_dir="llama-3.1-fine-tuned-model"

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=modules,
)


training_arguments = TrainingArguments(
    output_dir=output_dir,                    # directory to save and repository id
    num_train_epochs=1,                       # number of training epochs
    per_device_train_batch_size=1,            # batch size per device during training
    gradient_accumulation_steps=8,            # number of steps before performing a backward/update pass
    gradient_checkpointing=True,              # use gradient checkpointing to save memory
    optim="paged_adamw_32bit",
    logging_steps=1,
    learning_rate=2e-4,                       # learning rate, based on QLoRA paper
    weight_decay=0.001,
    fp16=True,
    bf16=False,
    max_grad_norm=0.3,                        # max gradient norm based on QLoRA paper
    max_steps=-1,
    warmup_ratio=0.03,                        # warmup ratio based on QLoRA paper
    group_by_length=False,
    lr_scheduler_type="cosine",               # use cosine learning rate scheduler
    report_to="wandb",                  # report metrics to w&b
    eval_strategy="steps",              # save checkpoint every epoch
    eval_steps = 0.2
)

trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    train_dataset=data["train"],
    eval_dataset=data["validation"],
    peft_config=peft_config,
    dataset_text_field="text",
    tokenizer=tokenizer,
    max_seq_length=2048,
    packing=False,
    dataset_kwargs={
    "add_special_tokens": False,
    "append_concat_token": False,
    }
)

In [ ]:
import wandb
wandb.init()
trainer.train()

In [ ]:
# prompt: download a folder

!zip -r /content/llama-3.1-fine-tuned-model.zip /content/llama-3.1-fine-tuned-model
from google.colab import files
files.download('/content/llama-3.1-fine-tuned-model.zip')


In [ ]:
wandb.finish()
model.config.use_cache = True

In [ ]:
# Save trained model and tokenizer
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

In [ ]:
!zip -r ./llama-3.1-fine-tuned-model

In [ ]:
y_pred = predict(data["test"].to_pandas(), model, tokenizer)
evaluate(data["test"].to_pandas()["sentimet"], y_pred)

In [ ]:
!unzip llama-3.1-fine-tuned1-model.zip

In [ ]:
from peft import PeftModel

In [ ]:
# Merge adapter with base model
fine_tuned_model = "./llama-3.1-fine-tuned-model/checkpoint-250"

model = PeftModel.from_pretrained(model, fine_tuned_model)
model = model.merge_and_unload()


In [ ]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto",
)
print(data["test"].to_pandas()["text"].iloc[1])
outputs = pipe(data["test"].to_pandas()["text"].iloc[1], max_new_tokens=2, do_sample=True, temperature=0.1)
print(outputs[0]["generated_text"].split("label: ")[-1].strip())



In [ ]:
y_pred = predict(data["test"].to_pandas().iloc[0:100], model, tokenizer)
evaluate(data["test"].to_pandas()["sentiment"].iloc[0:100], y_pred)

In [ ]:
# push to huggingface the model
model.push_to_hub("Materaili/llama-3.1-fine-tuned-model", use_auth_token=True, commit_message="Base test1 250 2k ")

